#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [3]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_1_4')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-c2f0ce56-f50b-475b-a0ee-2a086c2b11e6/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [4]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-c2f0ce56-f50b-475b-a0ee-2a086c2b11e6/bin'

# Construct the command
#command = [
#    './bin/neo4j-admin', 'database', 'import', 'full',
#    '--nodes=import/nodes.csv',
#    '--relationships=import/relationships.csv', 'neo4j'
#]

command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
Neo4j version: 5.12.0
Importing the contents of these files into C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-c2f0ce56-f50b-475b-a0ee-2a086c2b11e6\data\databases\neo4j:
Nodes:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-c2f0ce56-f50b-475b-a0ee-2a086c2b11e6\import\nodes.csv

Relationships:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-c2f0ce56-f50b-475b-a0ee-2a086c2b11e6\import\relationships.csv


Available resources:
  Total machine memory: 31.86GiB
  Free machine memory: 12.81GiB
  Max heap memory : 910.5MiB
  Max worker threads: 8
  Configured max memory: 10.81GiB
  High parallel IO: true

Cypher type normalization is enabled (disable with --normalize-types=false):
  Property type of 'WaveNumber' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-c2f0ce56-f50b-475b-a0ee-2a086c2b11e6\import\nodes.csv
  Property type of 'Absorbance' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\re

Run and connect to the Neo4j Database

In [2]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [3]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        SAMPLE: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        },
        GRID: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [45]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [46]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [47]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [48]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [49]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [50]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [51]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [14]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [4]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'DIST',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [52]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [54]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'id', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'id', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'id', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'id', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'id', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'id', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'id', 'Louvain'])

In [ ]:
pagerank_df = pagerank_df.groupby("name").mean().reset_index()

In [65]:
degree_df = degree_df.groupby("name").mean().reset_index()
eigenvector_df = eigenvector_df.groupby("name").mean().reset_index()
articlerank_df = articlerank_df.groupby("name").mean().reset_index()
eigenvector_df = eigenvector_df.groupby("name").mean().reset_index()
leiden_df = leiden_df.groupby("name").first().reset_index()
louvain_df = louvain_df.groupby("name").first().reset_index()

In [55]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['id', 'name'], how='left')

In [56]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210304-2-20,2554528,1.472942,346.263893,0.007547,0.232032,7,33118
1,210407-2-08,5289652,1.458982,684.913033,0.007900,0.301263,2,30039
2,210304-2-42,2612504,1.434791,341.089958,0.007298,0.230766,7,33118
3,210304-2-16,2543947,1.429760,290.571562,0.007041,0.219674,7,33118
4,210304-2-16,2543999,1.427356,290.504637,0.007031,0.219649,7,33118
...,...,...,...,...,...,...,...,...
51922,201210-1-22,57890,0.214485,25.412934,0.000221,0.155686,3,17080
51923,210114-1-38,361035,0.211477,9.794399,0.000170,0.152430,0,2445
51924,210114-2-34,484813,0.202604,32.973024,0.000282,0.157219,3,51517
51925,210414-3-34,5607088,0.202541,19.781042,0.000215,0.154603,2,22868


In [5]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'id', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,210504-2-42,6475071,0.191362,0.032645,0.086509,-0.109003,0.108920,0.203894,-0.167446,-0.006544,...,-0.179352,-0.053702,0.185285,0.067063,-0.050940,-0.055891,-0.099913,-0.114239,0.037986,-0.363996
1,210504-2-42,6475207,-0.168092,0.184825,-0.079223,-0.086608,-0.180025,0.354108,-0.032210,0.089148,...,-0.333258,-0.284970,-0.213477,0.041153,0.196321,0.244323,-0.039498,0.212122,0.057151,0.113711
2,210504-2-42,6475293,0.718534,-0.021204,0.143344,-0.363337,-0.088228,0.371912,0.256469,0.105800,...,0.102124,0.053059,-0.043240,0.100819,0.111246,0.181622,-0.022008,0.245993,0.097978,-0.363897
3,210504-2-42,6475365,0.720406,-0.019195,0.137163,-0.361034,-0.093582,0.376271,0.255695,0.106938,...,0.096277,0.054562,-0.043456,0.098588,0.119828,0.184937,-0.026262,0.249169,0.104143,-0.352928
4,210504-2-42,6475401,0.261929,0.061759,0.114241,0.020429,-0.081359,-0.240090,-0.008181,-0.149371,...,0.145015,-0.037697,0.170796,-0.011445,0.159523,-0.036042,-0.034843,0.007876,-0.014505,-0.211071


In [6]:
fastRP_df

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,210504-2-42,6475071,0.191362,0.032645,0.086509,-0.109003,0.108920,0.203894,-0.167446,-0.006544,...,-0.179352,-0.053702,0.185285,0.067063,-0.050940,-0.055891,-0.099913,-0.114239,0.037986,-0.363996
1,210504-2-42,6475207,-0.168092,0.184825,-0.079223,-0.086608,-0.180025,0.354108,-0.032210,0.089148,...,-0.333258,-0.284970,-0.213477,0.041153,0.196321,0.244323,-0.039498,0.212122,0.057151,0.113711
2,210504-2-42,6475293,0.718534,-0.021204,0.143344,-0.363337,-0.088228,0.371912,0.256469,0.105800,...,0.102124,0.053059,-0.043240,0.100819,0.111246,0.181622,-0.022008,0.245993,0.097978,-0.363897
3,210504-2-42,6475365,0.720406,-0.019195,0.137163,-0.361034,-0.093582,0.376271,0.255695,0.106938,...,0.096277,0.054562,-0.043456,0.098588,0.119828,0.184937,-0.026262,0.249169,0.104143,-0.352928
4,210504-2-42,6475401,0.261929,0.061759,0.114241,0.020429,-0.081359,-0.240090,-0.008181,-0.149371,...,0.145015,-0.037697,0.170796,-0.011445,0.159523,-0.036042,-0.034843,0.007876,-0.014505,-0.211071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51922,210504-2-42,6474732,-0.039752,-0.110337,-0.015146,-0.233646,-0.046195,0.223001,-0.100740,-0.224448,...,-0.064032,-0.118816,-0.121177,0.039586,0.192024,0.061414,-0.107679,0.335679,0.012106,-0.055034
51923,210504-2-42,6474822,-0.130612,-0.052813,-0.022823,0.175424,-0.309627,0.173640,-0.122262,-0.064900,...,0.000489,0.184656,-0.113247,0.177339,-0.319109,-0.133034,-0.006641,-0.034662,0.298174,0.334729
51924,210504-2-42,6474890,0.054971,-0.066047,-0.018402,-0.004523,0.294876,0.241080,-0.055914,-0.152232,...,-0.231079,0.014013,-0.051438,-0.004519,-0.007952,-0.075740,0.184617,-0.064193,0.135715,-0.373941
51925,210504-2-42,6474916,0.067644,-0.062079,-0.007803,0.000869,0.291397,0.239070,-0.057646,-0.162404,...,-0.233703,0.017865,-0.052485,-0.007645,-0.010926,-0.073265,0.190266,-0.062037,0.134607,-0.371175


Delete the projection

In [87]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [88]:
close_driver()

In [58]:
status_df

,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot,Node_ID:ID
0,201210-1-00,618.90662,1842.5710,Normal,57.732793,181.9741,"(12, 1)",613
1,201210-1-00,696.81091,1851.9185,Normal,33.525178,193.5749,"(13, 1)",727
2,201210-1-00,901.13898,1746.4041,Normal,107.305604,157.9830,"(18, 1)",1026
3,201210-1-00,996.12756,1702.7238,Normal,26.861586,115.9938,"(19, 1)",1165
4,201210-1-00,1542.14120,1475.0056,Normal,15.749470,87.6968,"(30, 1)",1964
...,...,...,...,...,...,...,...,...
51922,210526-3-49,450.79727,1482.5781,Hyperglycemia,11.954149,98.2019,"(9, 1)",8008132
51923,210526-3-49,520.50116,1482.3167,Hyperglycemia,13.875621,86.4581,"(10, 1)",8008234
51924,210526-3-49,705.01141,1507.8912,Hyperglycemia,54.718502,125.7380,"(14, 1)",8008504
51925,210526-3-49,1000.22780,1505.2144,Hyperglycemia,18.241825,110.7786,"(20, 1)",8008936


In [57]:
status_df = pd.read_csv('../../data/graph_1_4/nodes.csv')

In [10]:
fastRP_df['id'] = fastRP_df['id'].astype(int)

In [59]:
df['id'] = df['id'].astype(int)

In [65]:
graph_df = pd.merge(df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
graph_df.drop(columns=["Node_ID:ID"], inplace=True)

In [40]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
fast_graph_df.drop(columns=["Node_ID:ID"], inplace=True)

In [66]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)
graph_df.drop(columns=['SpecID_y','Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)

In [41]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [42]:
fast_graph_df.drop(columns=['SpecID_y', 'WaveNumber:float', 'Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
fast_graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)

In [67]:
graph_df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,WaveNumber:float,Status
0,210304-2-20,2554528,1.472942,346.263893,0.007547,0.232032,7,33118,1028.92940,Hypoglycemia
1,210407-2-08,5289652,1.458982,684.913033,0.007900,0.301263,2,30039,1024.82910,Normal
2,210304-2-42,2612504,1.434791,341.089958,0.007298,0.230766,7,33118,1033.02970,Hypoglycemia
3,210304-2-16,2543947,1.429760,290.571562,0.007041,0.219674,7,33118,1000.91110,Hypoglycemia
4,210304-2-16,2543999,1.427356,290.504637,0.007031,0.219649,7,33118,1036.44640,Hypoglycemia
...,...,...,...,...,...,...,...,...,...,...
51922,201210-1-22,57890,0.214485,25.412934,0.000221,0.155686,3,17080,1946.01370,Normal
51923,210114-1-38,361035,0.211477,9.794399,0.000170,0.152430,0,2445,227.33485,Normal
51924,210114-2-34,484813,0.202604,32.973024,0.000282,0.157219,3,51517,1982.23230,Normal
51925,210414-3-34,5607088,0.202541,19.781042,0.000215,0.154603,2,22868,1869.47610,Normal


In [68]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"

# Set the bin size
bin_size = 25

# Add a "Bin" column to the DataFrame
graph_df['Bin'] = graph_df['WaveNumber:float'].apply(lambda x: calculate_bin_interval(x, bin_size))

In [69]:
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = graph_df.pivot_table(index='SpecID', columns='Bin', values=['PageRank', 'DegreeCentrality', 'EigenvectorCentrality', 'ArticleRank', 'Leiden', 'Louvain'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = graph_df[['SpecID', 'Status']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)

C:\Users\stang\AppData\Local\Temp\ipykernel_29604\286844324.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  peak_bins.fillna(False, inplace=True)


In [72]:
graph_bins = peak_bins.reset_index()

In [74]:
peak_bins

,ArticleRank_1000-1025,ArticleRank_1025-1050,ArticleRank_1050-1075,ArticleRank_1075-1100,ArticleRank_1100-1125,ArticleRank_1125-1150,ArticleRank_1150-1175,ArticleRank_1175-1200,ArticleRank_1200-1225,ArticleRank_1225-1250,...,PageRank_775-800,PageRank_800-825,PageRank_825-850,PageRank_850-875,PageRank_875-900,PageRank_900-925,PageRank_925-950,PageRank_950-975,PageRank_975-1000,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,0.687047,False,False,0.922911,Normal
201210-1-01,False,False,False,False,False,0.281905,False,False,False,0.298914,...,False,False,False,False,0.819122,False,False,False,0.830504,Normal
201210-1-02,0.348318,False,False,False,False,0.274968,False,False,False,False,...,False,False,False,False,False,0.68636,False,False,False,Normal
201210-1-03,False,False,0.235851,False,False,False,0.213873,False,False,0.299763,...,False,False,False,False,False,0.755825,False,False,0.845046,Normal
201210-1-04,0.342233,False,False,False,False,False,False,0.263323,False,False,...,False,False,False,False,False,False,False,False,False,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-45,0.358607,False,False,False,0.294853,False,False,False,False,False,...,False,False,False,1.055476,False,False,False,False,False,Hyperglycemia
210526-3-46,0.365934,False,False,False,False,0.298666,False,False,False,False,...,False,False,False,1.082378,False,False,False,False,False,Hyperglycemia
210526-3-47,0.370506,False,False,False,0.301694,False,False,False,False,False,...,False,False,False,False,False,0.755558,False,False,False,Hyperglycemia


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = peak_bins.drop(['Status'], axis=1)
y = peak_bins['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.7496 +/- 0.0177
ExtraTreesClassifier Cross-Validation Accuracy: 0.7600 +/- 0.0187


In [100]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [101]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [102]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [103]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.7600671140939598


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.79      0.76      0.77       195
 Hypoglycemia       0.74      0.77      0.76       196
       Normal       0.75      0.75      0.75       205

     accuracy                           0.76       596
    macro avg       0.76      0.76      0.76       596
 weighted avg       0.76      0.76      0.76       596


Confusion Matrix:
[[148  22  25]
 [ 19 151  26]
 [ 21  30 154]]


In [104]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.7533557046979866


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.76      0.75      0.76       195
 Hypoglycemia       0.74      0.77      0.76       196
       Normal       0.76      0.74      0.75       205

     accuracy                           0.75       596
    macro avg       0.75      0.75      0.75       596
 weighted avg       0.75      0.75      0.75       596


Confusion Matrix:
[[147  23  25]
 [ 23 151  22]
 [ 24  30 151]]


In [105]:
get_feature_importances(rf_model, X)

,Feature,Importance
72,DegreeCentrality_1000-1025,0.010955
402,PageRank_250-275,0.010667
114,DegreeCentrality_250-275,0.009443
0,ArticleRank_1000-1025,0.009319
309,Louvain_1525-1550,0.008573
186,EigenvectorCentrality_250-275,0.008452
216,Leiden_1000-1025,0.008406
144,EigenvectorCentrality_1000-1025,0.008074
42,ArticleRank_250-275,0.007961
360,PageRank_1000-1025,0.007671


In [106]:
get_feature_importances(et_model, X)

,Feature,Importance
216,Leiden_1000-1025,0.008233
0,ArticleRank_1000-1025,0.007732
360,PageRank_1000-1025,0.007237
72,DegreeCentrality_1000-1025,0.007221
144,EigenvectorCentrality_1000-1025,0.006763
42,ArticleRank_250-275,0.006736
52,ArticleRank_500-525,0.006664
186,EigenvectorCentrality_250-275,0.006614
402,PageRank_250-275,0.006556
124,DegreeCentrality_500-525,0.006391


## FastRp

In [43]:
fast_graph_df.drop(columns=['id'], inplace=True)
fast_graph_df = fast_graph_df.groupby(['SpecID', 'Status']).max().reset_index()

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['SpecID', 'Status'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.7063758389261745
Extra Trees Accuracy: 0.7214765100671141


Fast rp mixed with traditional graph metrics

In [80]:
joined_graph = pd.merge(graph_bins, fast_graph_df, on=['SpecID', 'Status'], how='inner')

In [81]:
joined_graph

,SpecID,ArticleRank_1000-1025,ArticleRank_1025-1050,ArticleRank_1050-1075,ArticleRank_1075-1100,ArticleRank_1100-1125,ArticleRank_1125-1150,ArticleRank_1150-1175,ArticleRank_1175-1200,ArticleRank_1200-1225,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,False,False,False,False,False,False,False,False,False,...,0.421818,0.058655,0.178254,0.400232,0.072271,0.212378,0.337148,0.082718,0.143080,0.238950
1,201210-1-01,False,False,False,False,False,0.281905,False,False,False,...,0.291290,0.245867,0.372358,0.409377,0.274985,0.265400,0.344211,0.210164,0.380963,0.296081
2,201210-1-02,0.348318,False,False,False,False,0.274968,False,False,False,...,0.419659,0.318539,0.300556,0.181029,0.303717,0.316564,0.187152,0.143367,0.294984,0.355791
3,201210-1-03,False,False,0.235851,False,False,False,0.213873,False,False,...,0.313509,0.145580,0.185954,0.322233,0.319374,0.251014,0.385550,0.352416,0.243318,0.345959
4,201210-1-04,0.342233,False,False,False,False,False,False,0.263323,False,...,0.310170,0.194700,0.077513,0.138481,0.303800,0.315087,0.181740,0.132249,0.279425,0.352954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974,210526-3-45,0.358607,False,False,False,0.294853,False,False,False,False,...,0.274286,0.162695,0.172878,0.120871,0.221152,0.236208,0.370108,0.223027,0.331705,0.118262
2975,210526-3-46,0.365934,False,False,False,False,0.298666,False,False,False,...,0.274249,0.060969,0.283955,0.123554,0.399709,0.270380,-0.037927,0.224620,0.058218,0.117299
2976,210526-3-47,0.370506,False,False,False,0.301694,False,False,False,False,...,0.428799,0.165718,0.414559,0.323750,0.399434,0.270567,0.092501,0.253540,0.243880,0.041564
2977,210526-3-48,0.374132,False,False,False,False,0.302655,False,False,False,...,0.426540,0.165567,0.414956,0.127344,0.399662,0.270782,0.092976,0.253527,0.152573,0.117076


In [82]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.7533557046979866
Extra Trees Accuracy: 0.7718120805369127
